In [1]:
//#!value --name datasource --from-file "./waterConsumptionDataset.csv"

In [2]:
//#!share --from value datasource
//datasource

In [3]:
#r "nuget: Microsoft.Data.Analysis"
using Microsoft.Data.Analysis;
//var dataframe = DataFrame.LoadCsvFromString(datasource);

Installed Packages Microsoft.Data.Analysis, 0.19.1

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

In [4]:
using System.IO;

//Extracting file names of all the samples files
string directoryPath = "./samples";
DirectoryInfo d = new DirectoryInfo(directoryPath);
string[] files = d.GetFiles("*.txt").Select(f => f.Name).ToArray<string>();
files

index,value
0,2022-10-14-02-47-39-489-empty-no-movement.txt
1,2022-10-14-02-48-13-077-empty-lift-put-back-no-drinking.txt
2,2022-10-14-02-48-35-890-empty-lift-put-back-no-drinking.txt
3,2022-10-14-02-49-00-368-empty-lift-put-back-no-drinking.txt
4,2022-10-14-02-49-28-861-empty-lift-put-back-no-drinking.txt
5,2022-10-14-02-50-25-069-empty-no-movement.txt
6,2022-10-14-02-50-38-604-empty-lift-put-back-no-drinking.txt
7,2022-10-14-02-51-16-876-filling-from-empty-while-on-weight.txt
8,2022-10-14-02-52-07-118-adding-water-while-on-weight.txt
9,2022-10-14-02-52-29-994-adding-water-while-on-weight.txt


In [5]:
//defining row object
public record sourceDataRow(int ActionId, DateTime Time, double Weight, double AccX, double AccY, double AccZ){};

//converting timestamp format
DateTime CreateFromDateAndSpan(object date, object span)
{
    var adjustedSpan = span.ToString().Split(":").Select(e => int.Parse(e)).ToArray();
    
    return DateTime.Parse(date.ToString()).Add(new TimeSpan(0,adjustedSpan[0], adjustedSpan[1], adjustedSpan[2],adjustedSpan[3]));
}

//reading data from files 
IEnumerable<sourceDataRow> ScanDataSetsFromFiles(string directoryPath, params string[] files){
    int actionId = 0;
    foreach(var data in files){
        var source = DataFrame.LoadCsv(directoryPath + "/" + data);
        actionId += 1;
        foreach(var row in source.Rows){
            var time = CreateFromDateAndSpan(row[0],row[1]);
            var weight = Convert.ToDouble( row[2] );
            var accX = Convert.ToDouble( row[3] );
            var accY = Convert.ToDouble( row[4] );
            var accZ = Convert.ToDouble( row[5] );
            yield return new sourceDataRow(actionId, time, weight, accX, accY, accZ);
        }
    }
}

In [6]:
//defining destination dataframe format
var destination = new DataFrame();
var actionId = new PrimitiveDataFrameColumn<int>("ActionId");
var time = new PrimitiveDataFrameColumn<DateTime>("Time");
var windowDuration = new PrimitiveDataFrameColumn<TimeSpan>("WindowDuration");
var weight = new PrimitiveDataFrameColumn<double>("Weight");
var accX = new PrimitiveDataFrameColumn<double>("AccX");
var accY = new PrimitiveDataFrameColumn<double>("AccY");
var accZ = new PrimitiveDataFrameColumn<double>("AccZ");

destination.Columns.Add(actionId); //id of the action
destination.Columns.Add(time); //action time window start
destination.Columns.Add(windowDuration); //action time window duration
destination.Columns.Add(weight); //weight delta
destination.Columns.Add(accX); //avg accX
destination.Columns.Add(accY); //avg accY
destination.Columns.Add(accZ); //avg accZ

In [7]:
//creating destination dataset by calculating aggregation for weight and acceleration measures 
var window = new List<sourceDataRow>();
var currentAction = 1;
var timeWindow = new DateTime();

foreach (var row in ScanDataSetsFromFiles(directoryPath, files))
{    
    if (row.ActionId > currentAction)
    {
        destination.Append(new object[]{currentAction, window.First().Time,  window.Last().Time - window.First().Time, window.Last().Weight - window.First().Weight,  window.Select(w => w.AccX).Average(), window.Select(w => w.AccY).Average(), window.Select(w => w.AccZ).Average()}, true);
        currentAction = row.ActionId;
        window = new List<sourceDataRow>(){ row };
    }else{
        window.Add(row);
    }
}
destination.Append(new object[]{currentAction, window.First().Time, window.Last().Time - window.First().Time, window.Last().Weight - window.First().Weight,  window.Select(w => w.AccX).Average(), window.Select(w => w.AccY).Average(), window.Select(w => w.AccZ).Average()}, true);

destination.Display();

index,ActionId,Time,WindowDuration,Weight,AccX,AccY,AccZ
0,1,2022-10-14 02:47:36Z,00:00:03.1220000,-0.20000000298023224,-0.0441176477162277,0,1
1,2,2022-10-14 02:48:06Z,00:00:06.6790000,-0.7999999970197678,-0.008333333043588532,-0.07361111189756128,0.9833333326710595
2,3,2022-10-14 02:48:30Z,00:00:05.0860000,-0.6000000238418579,-0.0036363634196194734,-0.04000000086697665,0.9909090898253701
3,4,2022-10-14 02:48:52Z,00:00:07.8860000,0.6000000238418579,0.05294117725947324,-0.11882352864041049,0.90117647227119
4,5,2022-10-14 02:49:17Z,00:00:11.6280000,-0.20000001788139343,0.029600001037120818,-0.10080000025033951,0.9343999986648559
5,6,2022-10-14 02:50:20Z,00:00:04.4280000,-0.699999988079071,-0.0437500006519258,0,1
6,7,2022-10-14 02:50:34Z,00:00:03.9090000,-0.5,-0.07380952508676619,-0.014285714498588018,1
7,8,2022-10-14 02:51:05Z,00:00:11.1590000,147.70000612735748,-0.08083333453784386,0,1.0008333335320154
8,9,2022-10-14 02:51:56Z,00:00:10.5980000,77.39999389648438,-0.09736842250353411,0,1
9,10,2022-10-14 02:52:21Z,00:00:07.9040000,70.79998779296875,-0.09529411906705183,0,1


In [8]:
/*#r "nuget: Plotly.NET.Interactive"
#r "nuget: Plotly.NET.CSharp"
using Plotly.NET.CSharp;

Chart.Bar<double, string, string>(

    values: new double[] { 1, 2 , 4, 34, 65},
    Keys: new string[] { "1", "2" , "4", "34", "65"}

)

.WithTraceInfo("Hello from C#", ShowLegend: true)

.WithXAxisStyle<double, double, string>(Title: Plotly.NET.Title.init("xAxis"))

.WithYAxisStyle<double, double, string>(Title: Plotly.NET.Title.init("yAxis"))

Chart.Point<double, double, string>(

    x: new double[] { 1, 2 },

    y: new double[] { 5, 10 }

)

.WithTraceInfo("Hello from C#", ShowLegend: true)

.WithXAxisStyle<double, double, string>(Title: Plotly.NET.Title.init("xAxis"))

.WithYAxisStyle<double, double, string>(Title: Plotly.NET.Title.init("yAxis"))*/

Installed Packages Plotly.NET.CSharp, 0.8.0